In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from torch.autograd import Variable
from torch.nn import Linear, ReLU, Flatten, Sequential, Conv1d, MaxPool1d, Module, LogSoftmax, NLLLoss, Dropout, MSELoss, Softmax
from torch.optim import Adam, SGD
from torch.utils.data import Dataset

In [2]:
activity_encode = {
        "LAYING":0,
       "SITTING":1,
       "STANDING":2,
        "WALKING":3,
       "WALKING_DOWNSTAIRS":4,
       "WALKING_UPSTAIRS":5
       }

In [3]:
class UCI_Dynamic_Dataset(Dataset):
    """UCI dataset."""

    def __init__(self, csv_file, root_dir, encode=None):

        data = pd.read_csv(csv_file)
        #print(data.columns )
        data['Activity'] = data['Activity'].map(encode)
        # self.data = self.data.reshape(5881, 1, 561)
        data, _ = [x for _, x in data.groupby(data['Activity'] > 2)]
        self.data_y = data['Activity'].values
        data = pd.DataFrame(data.drop(['Activity','subject'],axis=1))
        self.data_x = np.array(data)
        self.root_dir = root_dir
        # [batch, channels, features]
        self.data_x = self.data_x.reshape(len(self.data_x), 1, 561)
        self.data_x  = torch.from_numpy(self.data_x)
        self.data_y = self.data_y.astype(int)
        self.data_y = torch.from_numpy(self.data_y)

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        # input = self.data_x.iloc[idx, 1:]
        # label = self.data_y.iloc[idx, 1:]
        # #
        # # sample = {'item': item}
        # x = self.data_x[idx]
        # y = self.data_y[idx]
        return self.data_x[idx], self.data_y[idx]

In [4]:
train_data = UCI_Dynamic_Dataset(csv_file='train.csv', root_dir='',encode=activity_encode)
valid_data = UCI_Dynamic_Dataset(csv_file='valid.csv', root_dir='',encode=activity_encode)
test_data = UCI_Dynamic_Dataset(csv_file='test.csv', root_dir='',encode=activity_encode)

In [5]:
valid_data.__len__()

826

In [6]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

# for inputs, labels in trainloader:
#     row = inputs
#     cols = labels
#     break

In [7]:
class StaticCNN(Module):
    def __init__(self):
        super(StaticCNN, self).__init__()

        self.cnn_layers = Sequential(
            # a
            Conv1d(1, 30, kernel_size=tuple([3])), #stride=tuple([1])),
            ReLU(),
            Conv1d(30, 50, kernel_size=tuple([3])),
            ReLU(),
            Conv1d(50, 100, kernel_size=tuple([3])),
            ReLU(),
            # Flatten
            Flatten()
        )

        self.linear_layers = Sequential(
            Dropout(0.5),
            Linear(55500, 3),
            #ReLU(inplace=True),
            Softmax(dim=1)
        )

    # Defining the forward pass
    def forward(self, x):
        # print(x.shape)
        x = self.cnn_layers(x)
        # print(x.shape)
        # x = x.view(-1, 32*100*561)
        #x = Flatten(x)
        #print(x.shape)
        x = self.linear_layers(x)
        #print(x.shape)
        return x

In [8]:
#NLLL not MSE because
criterion = NLLLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
def train(model, epochs=5):
    step = 10
    """
    Function that trains model over the training set, and validates on the validation set
    :param model: model to be trained
    :param epochs: Times training will be done
    :return: trained model
    """
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = Adam(model.parameters(), lr=0.0001)
    model.to(device)

    '''
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        data = data.view(args.batch_size,  3 * 8 * 8)
        target = target.view(args.batch_size, -1)
        output = model(data)
        loss = F.nll_loss(output, target[0])
    '''
    train_losses, valid_losses = [], []
    for e in range(epochs):
        running_loss = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            optimizer.zero_grad()
            # print(inputs.shape)
            # inputs = inputs.view(32,  32* 1* 561)
            # labels = labels.view((32, 3))

            log_ps = model(inputs)
            # print("after")
            #print(log_ps.shape)
            #print(labels.max())
            #print(labels.min())
            #loss_train = criterion(log_ps, labels)
            loss = criterion(log_ps, labels)
            #loss = criterion(labels,log_ps.view(1, -1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        else:
            valid_loss = 0
            accuracy = 0

            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                for inputs, labels in validloader:
                    inputs, labels = inputs.to(device).float(), labels.to(device).long()
                    log_ps = model(inputs)
                    valid_loss += criterion(log_ps, labels)

                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))

            train_losses.append(running_loss/len(train_data))
            valid_losses.append(valid_loss/len(valid_data))

            if (e+1) % step == 0:
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss),
                      "Validation Loss: {:.3f}.. ".format(valid_loss),
                      "Validation Accuracy: {:.3f}".format(accuracy))

In [9]:
model = StaticCNN()
train(model, epochs=100)

Epoch: 10/100..  Training Loss: -121.501..  Validation Loss: -24.863..  Validation Accuracy: 25.212
Epoch: 20/100..  Training Loss: -123.260..  Validation Loss: -25.355..  Validation Accuracy: 25.562
Epoch: 30/100..  Training Loss: -125.205..  Validation Loss: -25.356..  Validation Accuracy: 25.438
Epoch: 40/100..  Training Loss: -125.717..  Validation Loss: -25.628..  Validation Accuracy: 25.750
Epoch: 50/100..  Training Loss: -125.639..  Validation Loss: -25.379..  Validation Accuracy: 25.462
Epoch: 60/100..  Training Loss: -126.228..  Validation Loss: -25.675..  Validation Accuracy: 25.736
Epoch: 70/100..  Training Loss: -126.173..  Validation Loss: -25.464..  Validation Accuracy: 25.500
Epoch: 80/100..  Training Loss: -126.489..  Validation Loss: -25.815..  Validation Accuracy: 25.875
Epoch: 90/100..  Training Loss: -126.576..  Validation Loss: -25.720..  Validation Accuracy: 25.719
Epoch: 100/100..  Training Loss: -126.857..  Validation Loss: -25.571..  Validation Accuracy: 25.618

In [10]:
def test(model):
    """
    Tests model over the testing set
    :param model: to be tested
    """
    test_loss = 0
    accuracy = 0

    # Turn off gradients for testing, saves memory and computations
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            log_ps = model(inputs)
            test_loss += criterion(log_ps, labels)

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))

    print("Testing Accuracy: {:.3f}".format(accuracy))
test(model)

Testing Accuracy: 47.083
